In [1]:
import os
DEBUG = True
try:
    print("Original working directory: %s"%str(original_wd)) # type: ignore
    """
        You only get her0 if you---by mistake or otherwise---are re-running this cell, 
        in which case the working should not be changed again.
    """
except NameError:
    original_wd = os.getcwd()
    os.chdir('../')
print("Current working directory: %s"%str(os.getcwd()))

Current working directory: /Users/lucada/Documents/qhack/qhack21


In [2]:
from utils.data_handling import load_data, get_combined_cov_pos
from utils.dataset import Dataset

In [14]:
data = Dataset(load_data())
train_data, test_data = data.get_train_and_test()

def get_y(cov):
    eigs, _ = np.linalg.eigh(cov)
    eigs = np.log(eigs)/20
    return eigs
    
train_x = [x[:2] for x in train_data]
train_y = [get_y(x[2]) for x in train_data]
test_x = [x[:2] for x in test_data]
test_y = [get_y(x[2]) for x in test_data]

Reading data... ok
Filtering dataset... ok
Computing covmats... ok
Returning dataset with 10384/2596 training/testing datapoints


In [17]:
from utils.linalg import so3_to_su2, to_so3
from encoding import encode
import pennylane.numpy as np
import pennylane as qml
import pandas as pd

In [13]:
selected_ind = [0, 1]
wires = np.arange(5 * len(selected_ind))
dev = qml.device('default.qubit', wires=wires)

def layer(params=None, count_only=False):
    """
    A layer in our variational circuit
    
    If `count_only` is true, the function returns the number
    of necessary parameters.
    If `params` is None, then a dummy circuit is built and
    `count_only` must be set to true
    """
    if params is None and not count_only:
        raise ValueError("params cannot be None unless count_only=True")
        
    param_ind = 0
    def get_params(n = 1):
        """
        hands out trainable parameters from `params`
        
        Returns new parameter(s) at each call (assumes there
        are enough parameters available). The optional param
        `n` specifies how many parameters should be returned
        """
        nonlocal param_ind
        ret = []
        for i in range(n):
            if count_only:
                ret.append(0)
            else:
                ret.append(params[param_ind])
            param_ind += 1
        return ret if len(ret) > 1 else ret[0]
    
    qml.templates.AngleEmbedding(get_params(5), wires[:5], rotation='X')
    qml.templates.AngleEmbedding(get_params(5), wires[:5], rotation='Z')
    qml.templates.AngleEmbedding(get_params(5), wires[:5], rotation='X')

    # entangle position t0 with t1
    for ind in np.arange(3):
        q0 = wires[ind]
        q1 = wires[ind+5]
        qml.CRY(get_params(),wires=[q0, q1])
    # entangle cov(t0) with pos(t1)
    for ind in np.arange(3, 5):
        q0 = wires[ind]
        for jnd in np.arange(5, 8):
            q1 = wires[jnd]
            qml.CRY(get_params(),wires=[q0,q1])
    # entangle cov(t1) with pos(t1)
    for ind in np.arange(8, 10):
        q0 = wires[ind]
        for jnd in np.arange(5, 8):
            q1 = wires[jnd]
            qml.CRY(get_params(),wires=[q0,q1])
    
    qml.templates.AngleEmbedding(get_params(3), wires[5:8], rotation='X')
    qml.templates.AngleEmbedding(get_params(3), wires[5:8], rotation='Z')
    qml.templates.AngleEmbedding(get_params(3), wires[5:8], rotation='X')
    
    if count_only:
        return param_ind

@qml.qnode(dev)
def circ(covs, params):
    for ind, cov in enumerate(covs):
        encode(cov, wires=wires[5*ind:5*(ind+1)])
    layer(params)
    return [qml.expval(qml.PauliZ(wires=i)) for i in wires[5:8]]

def model(covs, params):
    out = circ(covs, params)
    alphas = 2*np.arccos(np.sqrt(.5 * (out[:3] + 1)))
    return alphas

n_params = layer(count_only=True)
circ(train_x[0], np.random.rand(n_params))
print(circ.draw())

 0: ──RX(0.187)───RX(0.611)──RZ(0.81)────RX(0.434)──────────────────────────────────────╭C──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤     
 1: ──RX(0.314)───RX(0.23)───RZ(0.188)───RX(0.996)──────────────────────────╭C──────────│───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤     
 2: ──RX(0.462)───RX(0.798)──RZ(0.214)───RX(0.569)──────────────╭C──────────│───────────│───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤     
 3: ──H──────────╭C──────────RZ(0.05)────RX(2.8)────RZ(

In [21]:
params = np.zeros(n_params)

opt = qml.AdamOptimizer(0.2)

# only take a small subset
train_x = train_x[:100]
train_y = train_y[:100]
test_x = test_x[:100]
test_y = test_y[:100]

def cost(x, y, params):
    out = model(x, params)
    return np.linalg.norm(out - y)**2

def accuracy(train_x, train_y, params):
    mse = 0
    for x,y in zip(train_x, train_y):
        mse += np.sqrt(cost(x, y, params)) / np.linalg.norm(y)
    return mse / len(train_x)

baseline_accuracy = accuracy(train_x, train_y, params)
print(f"Baseline error: {baseline_accuracy:.1%}")

n_epoch = 100

opti_error = pd.DataFrame(columns=['epoch', 'error', 'type'])

for epoch in np.arange(n_epoch):
    def obj_fn(params):
        loss = 0
        for x, y in zip(train_x, train_y):
            loss += cost(x, y, params)
        return loss
    params, current_loss = opt.step_and_cost(obj_fn, params)
    mse = accuracy(train_x, train_y, params)
    opti_error = opti_error.append({
        'epoch': epoch,
        'error': mse,
        'type': "training"
    }, ignore_index=True)
    test_error = accuracy(test_x, test_y, params)
    opti_error = opti_error.append({
        'epoch': epoch,
        'error': test_error,
        'type': "test"
    }, ignore_index=True)

    print(f"Epoch {epoch}: {mse:.1%} / testing error {test_error:.1%}")
    
train_accuracy = accuracy(train_x, train_y, params)
print(f"Final training error: {train_accuracy:.1%}")

Baseline error: 14.2%
Epoch 0: 67.4% / testing error 60.0%
Epoch 1: 62.9% / testing error 55.6%
Epoch 2: 39.7% / testing error 32.9%
Epoch 3: 21.9% / testing error 16.1%
Epoch 4: 25.5% / testing error 19.5%
Epoch 5: 29.5% / testing error 23.7%
Epoch 6: 32.2% / testing error 27.1%
Epoch 7: 33.9% / testing error 28.9%
Epoch 8: 34.4% / testing error 29.8%
Epoch 9: 34.0% / testing error 29.7%
Epoch 10: 29.7% / testing error 25.4%
Epoch 11: 27.0% / testing error 22.7%
Epoch 12: 28.7% / testing error 24.0%
Epoch 13: 28.2% / testing error 23.4%
Epoch 14: 26.6% / testing error 21.8%
Epoch 15: 24.6% / testing error 19.8%
Epoch 16: 22.9% / testing error 18.4%
Epoch 17: 23.1% / testing error 18.8%
Epoch 18: 24.0% / testing error 19.7%
Epoch 19: 23.4% / testing error 18.9%
Epoch 20: 21.9% / testing error 17.1%
Epoch 21: 21.0% / testing error 16.0%
Epoch 22: 21.1% / testing error 15.9%
Epoch 23: 21.7% / testing error 16.4%
Epoch 24: 21.6% / testing error 16.4%
Epoch 25: 22.3% / testing error 17.3%


In [ ]:
import seaborn as sns

sns.relplot(data=opti_error, x='epoch', y='error', hue='type')